In [167]:
#Import pandas, matplotlib.pyplot, and seaborn in the correct lines below

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [168]:
# Import data as .csv files

df_flight = pd.read_csv('flights.csv')
df_airport = pd.read_csv('airports.csv')
df_airline = pd.read_csv('airlines.csv')


C:\Users\wfgyi\AppData\Local\Temp\ipykernel_15560\1083225599.py:3: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_flight = pd.read_csv('flights.csv')


In [169]:
# Check the data information of the 3 imported csv files.
# df_flight.info()
df_airport.info()
df_airline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   IATA_CODE  322 non-null    object 
 1   AIRPORT    322 non-null    object 
 2   CITY       322 non-null    object 
 3   STATE      322 non-null    object 
 4   COUNTRY    322 non-null    object 
 5   LATITUDE   319 non-null    float64
 6   LONGITUDE  319 non-null    float64
dtypes: float64(2), object(5)
memory usage: 17.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   IATA_CODE  14 non-null     object
 1   AIRLINE    14 non-null     object
dtypes: object(2)
memory usage: 352.0+ bytes


In [170]:
df_cancel = df_flight[df_flight['CANCELLED'] == 1]
df_cancel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89884 entries, 32 to 5818843
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   YEAR                 89884 non-null  int64  
 1   MONTH                89884 non-null  int64  
 2   DAY                  89884 non-null  int64  
 3   DAY_OF_WEEK          89884 non-null  int64  
 4   AIRLINE              89884 non-null  object 
 5   FLIGHT_NUMBER        89884 non-null  int64  
 6   TAIL_NUMBER          75163 non-null  object 
 7   ORIGIN_AIRPORT       89884 non-null  object 
 8   DESTINATION_AIRPORT  89884 non-null  object 
 9   SCHEDULED_DEPARTURE  89884 non-null  int64  
 10  DEPARTURE_TIME       3731 non-null   float64
 11  DEPARTURE_DELAY      3731 non-null   float64
 12  TAXI_OUT             837 non-null    float64
 13  WHEELS_OFF           837 non-null    float64
 14  SCHEDULED_TIME       89879 non-null  float64
 15  ELAPSED_TIME         0 non-null  

In [171]:
# Delete all rows with no valid values.
all_null_cl = ['ELAPSED_TIME', 'AIR_TIME', 'WHEELS_ON', 'TAXI_IN', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
df_cancel = df_cancel[df_cancel.columns[~df_cancel.columns.isin(all_null_cl)]]

# Delete rows with over 90% missing data. 
df_cancel = df_cancel[df_cancel.columns[~df_cancel.columns.isin(['DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF'])]]

# Verify if all data entry 'YEAR' is 2015.  If so, delete 'YEAR' column.  
# Verify if all 'CANCELLED' cell has 1.  If so, delete 'CANCELLED' column.
# Delete 'DIVERTED' column.  
print(df_cancel['YEAR'].mean(), df_cancel['CANCELLED'].sum())
df_cancel = df_cancel[df_cancel.columns[~df_cancel.columns.isin(['YEAR', 'DIVERTED', 'CANCELLED'])]]

df_cancel.info()
df_cancel.head(3)

2015.0 89884
<class 'pandas.core.frame.DataFrame'>
Int64Index: 89884 entries, 32 to 5818843
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   MONTH                89884 non-null  int64  
 1   DAY                  89884 non-null  int64  
 2   DAY_OF_WEEK          89884 non-null  int64  
 3   AIRLINE              89884 non-null  object 
 4   FLIGHT_NUMBER        89884 non-null  int64  
 5   TAIL_NUMBER          75163 non-null  object 
 6   ORIGIN_AIRPORT       89884 non-null  object 
 7   DESTINATION_AIRPORT  89884 non-null  object 
 8   SCHEDULED_DEPARTURE  89884 non-null  int64  
 9   SCHEDULED_TIME       89879 non-null  float64
 10  DISTANCE             89884 non-null  int64  
 11  SCHEDULED_ARRIVAL    89884 non-null  int64  
 12  CANCELLATION_REASON  89884 non-null  object 
dtypes: float64(1), int64(7), object(5)
memory usage: 9.6+ MB


,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,CANCELLATION_REASON
32,1,1,4,AS,136,N431AS,ANC,SEA,135,205.0,1448,600,A
42,1,1,4,AA,2459,N3BDAA,PHX,DFW,200,120.0,868,500,B
68,1,1,4,OO,5254,N746SK,MAF,IAH,510,87.0,429,637,B


In [172]:
# Locate all the rows with a null value for 'SCHEDULED TIME'.
# print(df_cancel.loc[df_cancel.scheduled_time.isna(), axis=1])

In [173]:
# Replace NaN value in 'TAIL_NUMBER' column with ''.
df_cancel['TAIL_NUMBER'] = df_cancel['TAIL_NUMBER'].fillna('')

# Calculate missing values for 'SCHEDULED_TIME'.
# df_null = df_cancel.loc[df_cancel.isnull().any(axis=1)]
# df_null['SCHEDULED_TIME'] = df_null.apply(lambda row: ((row['SCHEDULED_ARRIVAL']-int(row['SCHEDULED_ARRIVAL']/100)*(100-60))-(row['SCHEDULED_DEPARTURE']-int(row['SCHEDULED_DEPARTURE']/100)*(100-60))), axis=1)
# df_cancel.loc[df_cancel.isnull().any(axis=1)] = df_cancel.apply(lambda row: ((row['SCHEDULED_ARRIVAL']-int(row['SCHEDULED_ARRIVAL']/100)*(100-60))-(row['SCHEDULED_DEPARTURE']-int(row['SCHEDULED_DEPARTURE']/100)*(100-60))), axis=1)


In [174]:
df_cancel.loc[[483174,619941,1820960,2031482,2034453]]

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,CANCELLATION_REASON
483174,2,1,7,NK,416,N627NK,FLL,IAG,2215,NaN,1176,107,B
619941,2,10,2,NK,416,N632NK,FLL,IAG,2215,NaN,1176,107,B
1820960,4,26,7,NK,720,N614NK,DEN,DFW,2059,NaN,641,2350,B
2031482,5,9,6,NK,324,N620NK,MCO,ACY,2130,NaN,852,2340,B
2034453,5,10,7,NK,368,N534NK,ORD,BWI,825,NaN,622,1118,C


In [175]:
df_airport = df_airport.drop(['LATITUDE','LONGITUDE', 'COUNTRY','AIRPORT'], axis=1)
df_airport.rename(columns = {'IATA_CODE':'ORIGIN_AIRPORT'}, inplace = True)

df_cancel = pd.merge(df_cancel, df_airport, how = 'left', on = ['ORIGIN_AIRPORT'])
df_cancel.rename(columns = {'CITY':'ORIGIN_CITY', 'STATE':'ORIGIN_STATE'}, inplace = True)

In [176]:
# Check the null values of 'ORIGIN_CITY' and 'ORIGIN_STATE'. Replace with 'UNKNOWN'.
df_cancel[df_cancel['ORIGIN_CITY'].isna()]
df_cancel = df_cancel.fillna('UNKNOWN')
df_cancel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89884 entries, 0 to 89883
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   MONTH                89884 non-null  int64 
 1   DAY                  89884 non-null  int64 
 2   DAY_OF_WEEK          89884 non-null  int64 
 3   AIRLINE              89884 non-null  object
 4   FLIGHT_NUMBER        89884 non-null  int64 
 5   TAIL_NUMBER          89884 non-null  object
 6   ORIGIN_AIRPORT       89884 non-null  object
 7   DESTINATION_AIRPORT  89884 non-null  object
 8   SCHEDULED_DEPARTURE  89884 non-null  int64 
 9   SCHEDULED_TIME       89884 non-null  object
 10  DISTANCE             89884 non-null  int64 
 11  SCHEDULED_ARRIVAL    89884 non-null  int64 
 12  CANCELLATION_REASON  89884 non-null  object
 13  ORIGIN_CITY          89884 non-null  object
 14  ORIGIN_STATE         89884 non-null  object
dtypes: int64(7), object(8)
memory usage: 11.0+ MB


In [177]:
df_airport.rename(columns = {'ORIGIN_AIRPORT':'DESTINATION_AIRPORT'}, inplace = True)

df_cancel = pd.merge(df_cancel, df_airport, how = 'left', on = ['DESTINATION_AIRPORT'])
df_cancel.rename(columns = {'CITY':'DESTINATION_CITY', 'STATE':'DESTINATION_STATE'}, inplace = True)

df_cancel = df_cancel.fillna('UNKNOWN')
df_cancel.info()
df_cancel.sample(20)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 89884 entries, 0 to 89883
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   MONTH                89884 non-null  int64 
 1   DAY                  89884 non-null  int64 
 2   DAY_OF_WEEK          89884 non-null  int64 
 3   AIRLINE              89884 non-null  object
 4   FLIGHT_NUMBER        89884 non-null  int64 
 5   TAIL_NUMBER          89884 non-null  object
 6   ORIGIN_AIRPORT       89884 non-null  object
 7   DESTINATION_AIRPORT  89884 non-null  object
 8   SCHEDULED_DEPARTURE  89884 non-null  int64 
 9   SCHEDULED_TIME       89884 non-null  object
 10  DISTANCE             89884 non-null  int64 
 11  SCHEDULED_ARRIVAL    89884 non-null  int64 
 12  CANCELLATION_REASON  89884 non-null  object
 13  ORIGIN_CITY          89884 non-null  object
 14  ORIGIN_STATE         89884 non-null  object
 15  DESTINATION_CITY     89884 non-null  object
 16  DEST

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,CANCELLATION_REASON,ORIGIN_CITY,ORIGIN_STATE,DESTINATION_CITY,DESTINATION_STATE
23303,2,16,1,DL,1083,N542US,ATL,RDU,2200,82.0,356,2322,B,Atlanta,GA,Raleigh,NC
65510,7,16,4,AA,307,N3MEAA,LGA,ORD,700,157.0,733,837,A,New York,NY,Chicago,IL
89018,12,29,2,EV,4917,N758EV,ATW,DTW,630,96.0,296,906,B,Appleton,WI,Detroit,MI
19878,2,9,1,EV,5405,N748EV,LGA,PIT,1845,102.0,335,2027,A,New York,NY,Pittsburgh,PA
25592,2,21,6,US,1777,,RSW,DCA,1920,139.0,892,2139,B,Ft. Myers,FL,Arlington,VA
62175,6,26,5,AA,2418,N504AA,DFW,JAX,2015,153.0,918,2348,B,Dallas-Fort Worth,TX,Jacksonville,FL
18729,2,8,7,VX,260,N633VA,LAS,JFK,2210,289.0,2248,559,C,Las Vegas,NV,New York,NY
62640,6,30,2,UA,1161,,BOS,EWR,944,86.0,200,1110,A,Boston,MA,Newark,NJ
5526,1,18,7,B6,1161,N519JB,LGA,PBI,1629,187.0,1035,1936,A,New York,NY,West Palm Beach,FL
49848,5,10,7,AA,1676,N567AA,DFW,BNA,2145,110.0,631,2335,B,Dallas-Fort Worth,TX,Nashville,TN


In [178]:
df_cancel.rename(columns = {'AIRLINE':'IATA_CODE'}, inplace = True)

In [179]:
df_cancel = pd.merge(df_cancel, df_airline, how = 'left', on = ['IATA_CODE'])
df_cancel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89884 entries, 0 to 89883
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   MONTH                89884 non-null  int64 
 1   DAY                  89884 non-null  int64 
 2   DAY_OF_WEEK          89884 non-null  int64 
 3   IATA_CODE            89884 non-null  object
 4   FLIGHT_NUMBER        89884 non-null  int64 
 5   TAIL_NUMBER          89884 non-null  object
 6   ORIGIN_AIRPORT       89884 non-null  object
 7   DESTINATION_AIRPORT  89884 non-null  object
 8   SCHEDULED_DEPARTURE  89884 non-null  int64 
 9   SCHEDULED_TIME       89884 non-null  object
 10  DISTANCE             89884 non-null  int64 
 11  SCHEDULED_ARRIVAL    89884 non-null  int64 
 12  CANCELLATION_REASON  89884 non-null  object
 13  ORIGIN_CITY          89884 non-null  object
 14  ORIGIN_STATE         89884 non-null  object
 15  DESTINATION_CITY     89884 non-null  object
 16  DEST

In [181]:
df_cancel.sample(10)

,MONTH,DAY,DAY_OF_WEEK,IATA_CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,CANCELLATION_REASON,ORIGIN_CITY,ORIGIN_STATE,DESTINATION_CITY,DESTINATION_STATE,AIRLINE
47566,4,25,6,MQ,2945,N650MQ,BMI,DFW,550,134.0,690,804,B,Bloomington,IL,Dallas-Fort Worth,TX,American Eagle Airlines Inc.
84167,12,20,7,OO,5494,N764SK,DEN,ASE,2150,50.0,125,2240,B,Denver,CO,Aspen,CO,Skywest Airlines Inc.
58908,6,17,3,WN,1984,N908WN,DAL,LBB,700,60.0,293,800,B,Dallas,TX,Lubbock,TX,Southwest Airlines Co.
34308,3,2,1,UA,1224,,LAX,HNL,1033,367.0,2556,1440,A,Los Angeles,CA,Honolulu,HI,United Air Lines Inc.
60033,6,20,6,MQ,3474,N818MQ,JFK,RDU,1455,126.0,427,1701,C,New York,NY,Raleigh,NC,American Eagle Airlines Inc.
15048,2,2,1,MQ,3463,N525MQ,LGA,RDU,1012,109.0,431,1201,C,New York,NY,Raleigh,NC,American Eagle Airlines Inc.
54451,6,3,3,OO,5316,N938SW,MRY,LAX,658,80.0,266,818,A,Monterey,CA,Los Angeles,CA,Skywest Airlines Inc.
23322,2,16,1,US,1820,,CLT,RSW,2210,109.0,600,2359,B,Charlotte,NC,Ft. Myers,FL,US Airways Inc.
20725,2,14,6,B6,122,N206JB,PBI,BOS,2100,170.0,1197,2350,B,West Palm Beach,FL,Boston,MA,JetBlue Airways
85645,12,27,7,MQ,3532,N520MQ,OKC,DFW,656,72.0,175,808,B,Oklahoma City,OK,Dallas-Fort Worth,TX,American Eagle Airlines Inc.
